# English Premier League 2019-2020

The following Notebook is a statistical analysis for the 2019-2020 English Premier League.

The __Player__ dataset was taken from Kaggle.com

Link to the dataset: https://www.kaggle.com/idoyo92/epl-stats-20192020

The __Match__ dataset was taken from Football-Data.co.uk
Link to the dataset: http://www.football-data.co.uk/englandm.php

## Dataset Column Images

In [22]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# pd.set_option('display.width', 500)
# pd.set_option('display.max_columns', 100)

In [23]:
# Read the data from the dataset into a dataframe
matchesDf = pd.read_csv("epl-dataset/epl2020.csv")

In [24]:
matchesDf.head()

,Unnamed: 0,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,teamId,ppda_cal,allowed_ppda,matchtime,tot_points,round,tot_goal,tot_con,Referee.x,HS.x,HST.x,HF.x,HC.x,HY.x,HR.x,AS.x,AST.x,AF.x,AC.x,AY.x,AR.x,B365H.x,B365D.x,B365A.x,HtrgPerc,AtrgPerc,matchDay
0,1,h,2.234560,0.842407,2.234560,0.842407,11,5,4,1,2.3863,w,2019-08-09 20:00:00,1,0,0,3,1.392153,Liverpool,5.942857,21.454545,20,3,1,4,1,M Oliver,15,7,9,11,0,0,12,5,9,2,2,0,1.14,10.0,19.00,0.466667,0.416667,Fri
1,2,a,0.842407,2.234560,0.842407,2.234560,5,11,1,4,0.4405,l,2019-08-09 20:00:00,0,0,1,0,-1.392153,Norwich,21.454545,5.942857,20,0,1,1,4,M Oliver,15,7,9,11,0,0,12,5,9,2,2,0,1.14,10.0,19.00,0.466667,0.416667,Fri
2,3,a,3.183770,1.200300,2.422640,1.200300,9,1,5,0,2.6312,w,2019-08-10 12:30:00,1,0,0,3,1.222340,Man City,6.935484,16.400000,12,3,1,5,0,M Dean,5,3,6,1,2,0,14,9,13,1,2,0,12.00,6.5,1.22,0.600000,0.642857,Sat
3,4,h,1.200300,3.183770,1.200300,2.422640,1,9,0,5,0.2522,l,2019-08-10 12:30:00,0,0,1,0,-1.222340,West Ham,16.400000,6.935484,12,0,1,0,5,M Dean,5,3,6,1,2,0,14,9,13,1,2,0,12.00,6.5,1.22,0.600000,0.642857,Sat
4,5,h,1.340990,1.598640,1.340990,1.598640,4,6,1,1,1.0172,d,2019-08-10 15:00:00,0,1,0,1,-0.257650,Bournemouth,13.600000,9.133333,15,1,1,1,1,K Friend,13,3,10,3,2,0,8,3,19,4,1,0,1.95,3.6,3.60,0.230769,0.375000,Sat


In the following result, we can see there are 576 rows

In [32]:
matchesDf.shape

(576, 45)

In [36]:
matchesDf.shape[0] / 2 # number of matches

288.0

In [39]:
# Display the last 5 rows
matchesDf.iloc[::-1].head()

,Unnamed: 0,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD,teamId,ppda_cal,allowed_ppda,matchtime,tot_points,round,tot_goal,tot_con,Referee.x,HS.x,HST.x,HF.x,HC.x,HY.x,HR.x,AS.x,AST.x,AF.x,AC.x,AY.x,AR.x,B365H.x,B365D.x,B365A.x,HtrgPerc,AtrgPerc,matchDay
575,576,h,2.412390,0.672448,1.651220,0.672448,8,2,4,0,2.5950,w,2020-03-09 20:00:00,1,0,0,3,0.978772,Leicester,5.379310,13.217391,20,53,29,58,28,M Oliver,15,7,15,9,2,0,4,1,12,0,1,0,1.36,5.25,8.0,0.466667,0.250000,Mon
574,575,a,0.672448,2.412390,0.672448,1.651220,2,8,0,4,0.2673,l,2020-03-09 20:00:00,0,0,1,0,-0.978772,Aston Villa,13.217391,5.379310,20,25,28,34,56,M Oliver,15,7,15,9,2,0,4,1,12,0,1,0,1.36,5.25,8.0,0.466667,0.250000,Mon
573,574,h,1.738900,0.585469,1.738900,0.585469,3,2,2,0,2.4269,w,2020-03-08 16:30:00,1,0,0,3,1.153431,Man Utd,14.709677,6.041667,16,45,29,44,30,M Dean,12,6,11,2,2,0,7,2,9,11,4,0,5.00,4.33,1.6,0.500000,0.285714,Sun
572,573,a,0.585469,1.738900,0.585469,1.738900,2,3,0,2,0.3539,l,2020-03-08 16:30:00,0,0,1,0,-1.153431,Man City,6.041667,14.709677,16,57,28,68,31,M Dean,12,6,11,2,2,0,7,2,9,11,4,0,5.00,4.33,1.6,0.500000,0.285714,Sun
571,572,a,0.445922,2.028830,0.445922,2.028830,3,9,0,4,0.2901,l,2020-03-08 14:00:00,0,0,1,0,-1.582908,Everton,19.086957,12.714286,14,37,29,37,46,K Friend,17,11,8,6,1,0,3,1,10,1,2,0,1.80,3.90,4.2,0.647059,0.333333,Sun


In [56]:
# Get the date for the last match
matchesDf.iloc[-1].date

'2020-03-09 20:00:00'

This has led to discovery that the dataset is incomplete. A full season should contain a total of __380__ matches.
The shape of the data above indicates there are only 288 (576 / 2) matches contained in the dataset. 
Upon manual inspection of the data, we can see that the date for the last match in the dataset occurs on the 9th of March 2020. This coincides with when the Premier League was postponed due to Covid. The League resumed several weeks later, but the dataset does not include the final matches.

### Check for missing values (null values)

Looking at the overall column stats provided by Kaggle, they provide some __total__ stats on each column within the dataset which allows us to identify any columns that may contain invalid data. And vice verse, we are able to identify columns that contain only valid data, eg. 1 and 0 or Y and N. This enables us to figure out which columns will need to be tested.

All boolean and numerical data columns do not indicate any invalid data when inspecting the column headings on Kaggle.com

This enables us to skip the need to check for null values within these columns.
We will still need to confirm there are no null values for the string value columns, such as the referee names, team names, day of the week, etc.

In [28]:
np.sum([matchesDf['Referee.x'].isnull()])

0